---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd
from time import strptime

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)

In [2]:
df

0           03/25/93 Total time of visit (in minutes):\n
1                         6/18/85 Primary Care Doctor:\n
2      sshe plans to move as of 7/8/71 In-Home Servic...
3                  7 on 9/27/75 Audit C Score Current:\n
4      2/6/96 sleep studyPain Treatment Pain Level (N...
                             ...                        
495    1979 Family Psych History: Family History of S...
496    therapist and friend died in ~2006 Parental/Ca...
497                         2008 partial thyroidectomy\n
498    sPt describes a history of sexual abuse as a c...
499    . In 1980, patient was living in Naples and de...
Length: 500, dtype: object

In [106]:
pattern1 = df.str.extract(r'0?(?P<month>\d{1,2})[/-]0?(?P<day>\d{1,2})[/-](?P<year>\d{2,4})').dropna()
def patch(item):
    if len(item['year']) == 2:
        item['year'] = '19'+item['year']
    return item
pattern1 = pattern1.apply(patch,axis=1)
pd.to_datetime(pattern1)

0     1993-03-25
1     1985-06-18
2     1971-07-08
3     1975-09-27
4     1996-02-06
         ...    
120   1998-12-22
121   1996-10-02
122   1990-11-05
123   1977-05-04
124   1996-02-27
Length: 125, dtype: datetime64[ns]

In [4]:
pattern2 = df.str.extract(r'(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\D+(?P<day>\d{1,2})\D{1,2}(?P<year>\d+)')
pattern2 = pattern2.dropna()
pattern2['month'] = pattern2.apply(lambda x:strptime(x['month'],'%b').tm_mon,axis=1)
pd.to_datetime(pattern2)

194   1990-04-11
195   2001-05-30
196   1994-02-18
197   1981-02-18
198   2013-10-11
199   1986-01-24
200   1978-07-26
201   1999-12-23
202   1989-05-15
203   1995-09-06
204   1976-03-10
205   1983-01-27
206   1990-10-23
207   2004-08-12
208   2012-09-01
209   1983-07-25
210   1989-08-11
211   1992-04-17
212   1999-07-24
213   1997-07-11
214   1974-09-10
215   1981-08-14
216   1988-11-11
217   2011-06-13
218   1989-05-14
219   1975-12-14
220   2012-06-25
221   1980-10-18
222   1998-05-15
223   1974-10-14
224   1998-07-25
225   1972-06-15
226   1991-01-07
227   2011-09-15
dtype: datetime64[ns]

In [5]:
pattern3 = df.str.extract(r'(?P<day>\d{1,2}) (?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\D+(?P<year>\d{4})')
pattern3 = pattern3.dropna()
pattern3['month'] = pattern3.apply(lambda x:strptime(x['month'],'%b').tm_mon,axis=1)
pd.to_datetime(pattern3)

125   2001-01-24
126   2004-09-10
127   1982-05-26
128   2002-06-28
129   1972-05-06
         ...    
189   1977-10-21
190   2000-08-10
191   1972-11-30
192   1993-05-06
193   1995-01-18
Length: 69, dtype: datetime64[ns]

In [59]:
pattern4 = df.str.extract(r'(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) (?P<day>\d{1,2})[a-z]{2}, (?P<year>\d{4})')
pattern4 = pattern4.dropna()
pattern4

,month,day,year


In [80]:
pattern5 =  df.str.extract(r'(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\D+(?P<year>\d{4})')
pattern5 = pattern5.dropna()
pattern5['day'] = 1
pattern5['month'] = pattern5.apply(lambda x:strptime(x['month'],'%b').tm_mon,axis=1)
pattern5 = pattern5.loc[set(pattern5.index) - set(pattern3.index)]
pd.to_datetime(pattern5).sort_index()

228   1985-09-01
229   2011-06-01
230   1986-05-01
231   2016-05-01
232   1977-07-01
         ...    
338   1998-04-01
339   2005-03-01
340   1980-05-01
341   2007-11-01
342   1976-03-01
Length: 115, dtype: datetime64[ns]

In [76]:
pattern6 = df.str.extract(r'0?(?P<month>\d{1,2})/(?P<year>\d{4})')
pattern6 = pattern6.dropna()
pattern6['day'] = 1

pattern6 = pattern6.loc[set(pattern6.index)-set(pattern1.index)]
pattern6
#pd.to_datetime(pattern6)

,month,year,day
343,6,1998,1
344,6,2005,1
345,10,1973,1
346,9,2005,1
347,3,1980,1
...,...,...,...
450,1,1994,1
451,12,2004,1
452,3,2003,1
453,7,1991,1


In [114]:
pattern7 = df.str.extract(r'(?P<year>\d{4})')
pattern7 = pattern7.dropna()
mask = set(pattern7.index)
for pattern in [pattern1,pattern2,pattern3,pattern4,pattern5,pattern6]:
    mask = mask - set(pattern.index)
pattern7 = pattern7.loc[mask]
pattern7 = pattern7[pattern7['year']<='3']
pattern7['day'] = 1
pattern7['month'] = 1
pattern7 = pattern7.sort_index()
pd.to_datetime(pattern7)

455   1984-01-01
456   2000-01-01
457   2001-01-01
458   1982-01-01
459   1998-01-01
460   2012-01-01
461   1991-01-01
462   1988-01-01
463   2014-01-01
464   2016-01-01
465   1976-01-01
466   1981-01-01
467   2011-01-01
468   1997-01-01
469   2003-01-01
470   1983-01-01
471   1999-01-01
472   2010-01-01
473   1975-01-01
474   1972-01-01
475   2015-01-01
476   1989-01-01
477   1994-01-01
478   1993-01-01
479   1996-01-01
480   2013-01-01
481   1974-01-01
482   1990-01-01
483   1995-01-01
484   2004-01-01
485   1987-01-01
486   1973-01-01
487   1992-01-01
488   1977-01-01
489   1985-01-01
490   2007-01-01
491   2009-01-01
492   1986-01-01
493   1978-01-01
494   2002-01-01
495   1979-01-01
496   2006-01-01
497   2008-01-01
498   2005-01-01
499   1980-01-01
dtype: datetime64[ns]

In [117]:
result = []
for pattern in [pattern1,pattern2,pattern3,pattern4,pattern5,pattern6,pattern7]:
    result += list(pattern.index)
len(result),result,set(result) - set(range(500))

(500,
 [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  194,
  195,
  196,
  197,
  198,
  199,
  200,
  201,
  202,
  203,
  204,
  205,
  206,
  207,
  208,
  209,
  210,
  211,
  212,
  213,
  214,
  215,
  216,
  217,
  218,
  219,
  220,
  221,
  222,
  223,
  224,
  225,
  226

In [129]:
pattern1_date = pd.to_datetime(pattern1)
pattern2_date = pd.to_datetime(pattern2)
pattern3_date = pd.to_datetime(pattern3)
pattern4_date = pd.to_datetime(pattern4)
pattern5_date = pd.to_datetime(pattern5)
pattern6_date = pd.to_datetime(pattern6)
pattern7_date = pd.to_datetime(pattern7)
date = pd.concat([pattern1_date,pattern2_date,pattern3_date,pattern4_date,
          pattern5_date,pattern6_date,pattern7_date])
date_rank = date.rank().astype('int64')

In [131]:
def date_sorter():
    pattern1 = df.str.extract(r'0?(?P<month>\d{1,2})[/-]0?(?P<day>\d{1,2})[/-](?P<year>\d{2,4})').dropna()
    def patch(item):
        if len(item['year']) == 2:
            item['year'] = '19'+item['year']
        return item
    pattern1 = pattern1.apply(patch,axis=1)
    
    pattern2 = df.str.extract(r'(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\D+(?P<day>\d{1,2})\D{1,2}(?P<year>\d+)')
    pattern2 = pattern2.dropna()
    pattern2['month'] = pattern2.apply(lambda x:strptime(x['month'],'%b').tm_mon,axis=1)

    pattern3 = df.str.extract(r'(?P<day>\d{1,2}) (?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\D+(?P<year>\d{4})')
    pattern3 = pattern3.dropna()
    pattern3['month'] = pattern3.apply(lambda x:strptime(x['month'],'%b').tm_mon,axis=1)

    pattern4 = df.str.extract(r'(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) (?P<day>\d{1,2})[a-z]{2}, (?P<year>\d{4})')
    pattern4 = pattern4.dropna()

    pattern5 =  df.str.extract(r'(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\D+(?P<year>\d{4})')
    pattern5 = pattern5.dropna()
    pattern5['day'] = 1
    pattern5['month'] = pattern5.apply(lambda x:strptime(x['month'],'%b').tm_mon,axis=1)
    pattern5 = pattern5.loc[set(pattern5.index) - set(pattern3.index)]

    pattern6 = df.str.extract(r'0?(?P<month>\d{1,2})/(?P<year>\d{4})')
    pattern6 = pattern6.dropna()
    pattern6['day'] = 1
    pattern6 = pattern6.loc[set(pattern6.index)-set(pattern1.index)]

    pattern7 = df.str.extract(r'(?P<year>\d{4})')
    pattern7 = pattern7.dropna()
    mask = set(pattern7.index)
    for pattern in [pattern1,pattern2,pattern3,pattern4,pattern5,pattern6]:
        mask = mask - set(pattern.index)
    pattern7 = pattern7.loc[mask]
    pattern7 = pattern7[pattern7['year']<='3']
    pattern7['day'] = 1
    pattern7['month'] = 1
    pattern7 = pattern7.sort_index()

    pattern1_date = pd.to_datetime(pattern1)
    pattern2_date = pd.to_datetime(pattern2)
    pattern3_date = pd.to_datetime(pattern3)
    pattern4_date = pd.to_datetime(pattern4)
    pattern5_date = pd.to_datetime(pattern5)
    pattern6_date = pd.to_datetime(pattern6)
    pattern7_date = pd.to_datetime(pattern7)
    date = pd.concat([pattern1_date,pattern2_date,pattern3_date,pattern4_date,
              pattern5_date,pattern6_date,pattern7_date])
    date_rank = date.rank().astype('int64')
    return date_rank

In [ ]:
pattern1_date = pd.to_datetime(pattern1)
pattern2_date = pd.to_datetime(pattern2)
pattern3_date = pd.to_datetime(pattern3)
pattern4_date = pd.to_datetime(pattern4)
pattern5_date = pd.to_datetime(pattern5)
pattern6_date = pd.to_datetime(pattern6)
pattern7_date = pd.to_datetime(pattern7)
date = pd.concat([pattern1_date,pattern2_date,pattern3_date,pattern4_date,pattern5_date,pattern6_date,pattern7_date])
date_rank = date.rank().astype('int64')

In coursera notebook, I experienced kernel failed error. I think It's because of memory failure. I changed the code right below.

In [ ]:
date = pd.concat([pattern1,pattern2,pattern3,pattern4,pattern5,pattern6,pattern7])
date = pd.to_datetime(date)
date_rank = date.rank().astype('int64')

This code works very well. It seems like that making many dateframes requires more memory than making few dataframes unless their data amounts are same.

In [134]:
date_sorter()

0      273
1      171
2        3
3       56
4      313
      ... 
495     97
496    409
497    430
498    403
499    107
Length: 500, dtype: int64